In [1]:
import timeit
from typing import Any, TypeVar
from numpy import array, ndarray, sqrt, square, sum, divide, multiply

Shape = TypeVar('Shape')
def cosine(first: list[float] | ndarray[Shape, float], second: list[float] | ndarray[Shape, float]) -> float:
  first: ndarray[Any, float] = array(first, dtype=float)
  second: ndarray[Any, float] = array(second, dtype=float)

  return divide(sum(multiply(first, second)), sqrt(multiply(sum(square(first)), sum(square(second)))))


print(cosine([1.0, 2.0, 3.0], [1.5, -0.7, -20]))
print(cosine([-10.0, 17.0, 2.0], [5.3, 12.0, -20]))
print(cosine([1.0, 2.0, 3.0], [1, -3000, 184]))
print(cosine([1.0, 2.0, 3.0], [1, 2, 3]))


-0.7977198918178164
0.23409628697705598
-0.48434715333575534
1.0


<div class="alert alert-block alert-success">
**Oczekiwany rezultat:** <br/>
<ul>
<li>-0.797719891818</li>
<li>0.234096286977</li>
<li>-0.484347153336</li>
<li>1.0</li>
</ul>
</div>

Za pomocą CountVectorizera stwórzmy reprezentację BOW dwóch krótkich dokumentów i sprawdźmy z użyciem stworzonej funkcji jakie jest ich podobieństwo. Uruchom poniższy kod:

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

doc1 = "Ala ma kota"
doc2 = "Ala ma pięknego puszystego kota"
docs = [doc1, doc2]

X_train_counts = vectorizer.fit_transform(docs).todense()

print("Dokumenty reprezentowane jako BOW. Dokumenty w wierszach, słowa w kolumnach")
print(X_train_counts)
print("\nPodobieńśtwo dokumentów to")
# tolist()[0] zamienia macierz 1xn na listę elementów 1xn
print(cosine(X_train_counts[0].tolist()[0], X_train_counts[1].tolist()[0]))


Dokumenty reprezentowane jako BOW. Dokumenty w wierszach, słowa w kolumnach
[[1 1 1 0 0]
 [1 1 1 1 1]]

Podobieńśtwo dokumentów to
0.7745966692414834


Kiedy dokumenty zawierają te same słowa podobieństwo zostaje całkiem nieźle obliczone, co jednak, jeśli zamiast takich samych słów znajdziemy wyrazy bliskoznaczne? Sprawdźmy - uruchom poniższy kod:

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

doc1 = "kot"
doc2 = "kotek"
docs = [doc1, doc2]

print("Dokumenty reprezentowane jako BOW. Dokumenty w wierszach, słowa w kolumnach")
X_train_counts = vectorizer.fit_transform(docs).todense()
print(X_train_counts)

print("\n\nPodobieńśtwo dokumentów to")
print(cosine(X_train_counts[0].tolist()[0], X_train_counts[1].tolist()[0]))

Dokumenty reprezentowane jako BOW. Dokumenty w wierszach, słowa w kolumnach
[[1 0]
 [0 1]]


Podobieńśtwo dokumentów to
0.0


Jak widzimy BOW nie radzi sobie z oceną podobieństwa synonimów. Dajmy zatem szansę embeddingom!

---
Embeddingi to nic innego jak osadzenie znaczenia słowa w n-wymiarowej przestrzeni tak, aby słowa podobne do siebie występowały blisko w tej n-wymiarowej przestrzeni. Możemy stworzyć je sami na podstawie dużego korpusu tekstu (co może być czasochłonne), z użyciem paczek takich jak: gensim (https://radimrehurek.com/gensim/) ale możemy również użyć "pretrenowanych" wektorów utworzonych już na jakimś korpusie, dostępnych np. pod adresem: (https://nlp.stanford.edu/projects/glove/). My wybierzemy opcję drugą - wykorzystania istniejących wektorów. <br/>

Embeddingi dostarczone przez zespół Stanforda to pliki tekstowe postaci:<br/>
słowo[SPACJA]wektor_liczb_oddzielonych_spacją_reprezentujących_znaczenie_slowa<br/>

Funkcja ładująca embeddingi do pamięci została już stworzona. <br/>**Uruchom poniższy kod, aby móc wykrzystać tę funkcję w kolejnych komórkach i ocenić podobieństwo wyrazów bliskoznacznych.** <span style="color:red">Uwaga: zmienna mapping bedzie wykorzystywana w kolejnych komorkach, więc aby była dla nich widoczna - poniższa komórka musi być koniecznie uruchomiona.</span>

In [4]:
import numpy as np

def load_embeddings(path):
  mapping = dict()

  with open(path, 'r', encoding='utf8') as file:
    for line in file:
      line = line.strip()
      split = line.split(" ")
      # stwórz słownik słowo -> wektor
      mapping[split[0]] = np.array(split[1:], dtype=float)
  return mapping

# ładujemy wektory o długości 50 liczb
mapping = load_embeddings('resources/glove.6B.50d.txt')

kot = mapping['cat']
kotek = mapping['kitten']
krzeslo = mapping['chair']

print(f"Podobieństwo między kotem a kotkiem: {cosine(kot, kotek)}")
print(f"Podobieństwo między kotem a krzesłem: {cosine(kot, krzeslo)}")

Podobieństwo między kotem a kotkiem: 0.6386305647068641
Podobieństwo między kotem a krzesłem: 0.29425297716624554


Widzimy, że embeddingi, które tutaj są dość krótkie (50 liczb opisujących każde słowo) dobrze oddają podobieństwo. Kot jest dużo bardziej podobny do kotka niż do krzesła!

# Zadanie 2: Embeddingi - wizualizacja i podobieństwo

Embeddingi są reprezentacją znaczenia słów w n-wymiarowej przestrzeni. Strona: http://projector.tensorflow.org próbuje zwizualizować tę przestrzeń poprzez rzutowanie pretrenowanych wektorów na przestrzeń 3-wymiarową.

W celu wykonania zadania, otwórz powyższą stronę i wykonaj podpunkty a) i b)  <br/>
**Zadanie 2a (0.5 punktu)**: Podaj 5 najbliższych słów do słowa "data" w domyślnie załadowanych embeddingach (Słowo, którego sąsiadów chcemy zlokalizować możemy wpisać w pole "Search" w górnej części po prawej stronie ekranu. Następnie, po wskazaniu słowa "data" ujrzymy listę najbardziej podobnych słów wraz z miarą podobieńtstwa. Listę najbardziej podobnych słów wraz z miarą odległości (wybierz cosinusową), zawrzyj w poniższym komentarzu.
<br/>
Uwaga - na stronie użyto odległości cosinusowej zamiast podobieństwa. Relacja pomiędzy obiema miarami jest bardzo prosta: odległość = 1 - podobieństwo

In [5]:
# 1. Słowo: register     Odległość: 0.554
# 2. Słowo: file         Odległość: 0.548
# 4. Słowo: instructions Odległość: 0.512
# 3. Słowo: files        Odległość: 0.528
# 5. Słowo: information  Odległość: 0.443

Jak widzimy, słowa, które są najbliższe słowu "data" to w tym przypadku wyrazy bliskoznaczne. <br/>
**Zadanie 2b (0.5 punktu)**: Podaj 5 najbliższych słów do słowa "red". Czy nadal mamy do czynienia z synonimami?
Odpowiedz na pytanie: jak można interpretować najbardziej podobne słowa(w jakim aspekcie są one podobne), skoro jak widać nie są to synonimy (przypomnij sobie zasadę działania embeddingów)? Odpowiedzi zawrzyj w komentarzach poniżej

In [6]:
# 1. Słowo: black  Odległość: 0.494
# 2. Słowo: green  Odległość: 0.398
# 3. Słowo: white  Odległość: 0.395
# 4. Słowo: yellow Odległość: 0.382
# 5. Słowo: blue   Odległość: 0.335

# Interpretacja najbliższych słów: Czarny często występuje w parze z kolorem czerwonym oraz jest często traktowany też jako przeciwieństwo niebieskiego, ale nadal są kolorami więc istnieje powiązanie między nimi

# Zadanie 3: Relacje między wektorami 

Embeddingi zawierają w sobie informacje o znaczeniu. Co więcej - są wektorami, przez co możemy wykonywać na nich operacje (dodawanie, odejmowanie,...). Sprawdźmy jakie efekty uzyskamy wykonując operacje na tych wektorach.
<br/>
Interesować nas będzie efekt operacji: $\vec{italy} - \vec{rome} + \vec{warsaw}$. Na co wskazywać będzie tak zdefiniowany wektor?<br/>
Ponieważ wynikiem tej operacji będzie nowy wektor, napiszmy funkcję, która sprawdzi jakie istniejące słowo leży najbliżej tego wektora.

**Zadanie 3:** Wypełnij funkcję get_most_similar() tak, aby dla zadanego wektora vec1, zwróciła słowo, którego wektor jest najbardziej podobny do wektora vec1 (do oceny podobieńśtwa wykorzystaj stworzoną w zad. 1 funkcję cosine). Parametrem embeddings jest słownik, który mapuje słowo na odpowiadający mu wektor.<br/>
Jakie słowo zostało wyznaczone jako najbliższe do obliczonego punktu?

In [7]:
from typing import TypeVar

Shape = TypeVar('Shape')
def get_most_similar(vector: ndarray[Shape, float], embeddings: dict[str, ndarray[Shape, float]]) -> str:
  return max(embeddings, key=lambda x: cosine(vector, embeddings[x]))

new_point = mapping['italy'] - mapping['rome'] + mapping['warsaw']
print(get_most_similar(new_point, mapping))

poland


In [8]:
print(new_point)

[-0.13434   -0.55924   -0.061269   0.12031   -0.059634   1.119708
  0.706171   0.47393   -0.74465   -1.045956   1.318756  -2.02057
 -0.42257    0.02422    0.35484    0.26507    0.615026   0.5488
 -0.46353    0.53853    0.871282  -0.857051  -0.85398    0.93783
 -0.346134  -1.4091     0.426467   0.11677   -0.25116    0.223392
  1.75266    0.9151     0.236205   0.06239   -1.33426   -0.178076
 -0.54741    1.24018   -0.19442   -0.35939    0.42025    0.1212903
  1.30291   -2.24267   -0.33217    0.9972    -0.14806    0.1206
 -0.4379    -0.53486  ]


Widzimy zatem, że wykonywanie operacji na embeddingach pozwala na uzyskanie bardzo ciekawych rezulatów. Jeśli od obiektu "Włochy" odejmiemy jego stolicę, a dodamy stolicę polski, to otrzymamy obiekt "Polska". Innymi słowy - odpowiadamy na pytanie: jakie słowo jest w takiej relacji do Polski, w jakiej jest Rzym do Włoch.

# Zadanie 4: Klasyfikacja

Okazuje się również, że embeddingi użyteczne są w klasyfikacji, skutecznie redukując nam liczbę cech a także rozwiązując problem rzadkiej reprezentacji. Wróćmy do zadania klasyfikacji spamu znanego z jednych z poprzenich laboratoriów. W celu decydowania czy mamy do czynienia ze spamem, czy może hamem, chcielibyśmy użyć klasyfikatora SVC, który jako cechy przyjmie embeddingi. <br/>

Ponieważ jednak embeddingi opisują pojedyncze słowa jako n-wymiarowe wektory, a w problemie klasyfikacji maili musimy reprezentować całe dokumenty w postaci wektorów - musimy zagregować informacje o wszystkich słowach w jednym wektorze cech.
<br/>

Jedną z zadziwiająco dobrze działających metod okazuje się być reprezentowanie całego dokumentu jako wektora, będącego "środkiem ciężkości" słów z których jest zbudowany. Wektor wynikowy jest wektorem n-wymiarowym (tak jak wektor każdego ze słów "składowych"), gdzie $i$-ta pozycja wektora posiada wartość będącą średnią arytmetyczną $i$-tych pozycji wektorów słów z danego dokumentu. UWAGA: może okazać się, że w pretrenowanych embeddingach słowo z dokumentu, który chcemy reprezentować jako wektor nie występuje. W takiej syturacji zignorujmy kompletnie to słowo (uznajmy, że go nie ma).
<br/>
<br/>
**Zadanie 4 (1 punkt)**: Zaimplementuj funkcję documents_to_ave_embeddings(docs, embeddings) [ linia 21 ] przyjmującej dwa parametry: 
<ol>
    <li>docs - lista dokumentów w formie tekstowej (lista stringów) do przetransformowania na wektory</li>
    <li>embeddings - mapowanie słowo -> wektor (embedding) z istniejącego modelu</li>
</ol>
Funkcja powinna zwrócić jedną zmienną, listę wektorów dokumentów, gdzie 1 dokument jest "średnim wektorem" wektorów słów z których jest zbudowany (jak w paragrafie powyżej treści zadania). Użyj tokenizatora z NLTK (word_tokenize) i przed stokenizowaniem - zamień teksty dokumentów na składające się z samych małych liter.

Rozważ użycie numpy.mean z odpowiednią wartością parametru axis do policzenia średniej (będzie łatwiej użyć gotowej funkcji niż ręcznie implementować).

In [66]:
from typing import Iterable
from numpy import vectorize
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas
import numpy as np
from nltk import word_tokenize
from sklearn.metrics import classification_report

# ------------------- WCZYTANIE DANYCH -----------

# wczytaj dane z pliku CSV
full_dataset = pandas.read_csv('resources/spam-emails.csv', encoding='utf-8')
# ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam", wykonujemy mapowanie tych wartości na liczby, aby móc wykonać klasyfikację.
full_dataset['label_num'] = full_dataset.label.map({'ham': 0, 'spam': 1})

# ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
np.random.seed(0)
# wylosuj 70% wierszy, które znajdą się w zbiorze treningowym
train_indices = np.random.rand(len(full_dataset)) < 0.7

# wybierz zbiór treningowy (70%)
train = full_dataset[train_indices]
# wybierz zbiór testowy (dopełnienie treningowego - 30%)
test = full_dataset[~train_indices]

def documents_to_ave_embeddings(docs: ndarray, embeddings: dict[str, ndarray]):
  return array([[array([*map(embeddings.get, filter(embeddings.__contains__, words))]).mean()]
                for words in map(word_tokenize, map(str.lower, docs))])

# ------------------- STWORZENIE PIPELINE'U -----------

train_transformed = documents_to_ave_embeddings(train['text'], mapping)
test_transformed = documents_to_ave_embeddings(test['text'], mapping)

# ------------------- TRANSFORMACJA I UCZENIE -----------

# zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym
classifier = SVC(C=1.0)
classifier.fit(train_transformed, train['label_num'])

# ------------------- OCENA KLASYFIKATORA -----------
accuracy = classifier.score(test_transformed, test['label_num'])
print(f"W zbiorze testowym {100. * accuracy}% przypadków zostało poprawnie zaklasyfikowanych!")
# testowanie klasyfikatora - szerokie podsumowanie uwzględniające miary: precision, recall, f1
print(classification_report(test['label_num'], classifier.predict(test_transformed)))

W zbiorze testowym 72.57844474761255% przypadków zostało poprawnie zaklasyfikowanych!
              precision    recall  f1-score   support

           0       0.73      0.97      0.83       517
           1       0.65      0.15      0.25       216

    accuracy                           0.73       733
   macro avg       0.69      0.56      0.54       733
weighted avg       0.71      0.73      0.66       733



# Zadanie 5:  Odległość Levensteina (poprawianie literówek)
Co prawda odległość Levensteina nie jest związana z koncepcją embeddingów, jednak do tej pory nie próbowaliśmy zastosować tej metody w praktyce. Ostatnie zadanie tych laboratoriów będzie dotyczyło właśnie odległości edycyjnej. 

**Zadanie 5 (1 punkt)**: Zaimplementuj funkcję get_closest_word która poprawi literówki z użyciem odległości Levensteina. Jeśli aktualny token występuje w słowniku (words_set) - nie powinien zostać korygowany, jeśli zaś nie występuje - powinien być zamieniony na słowo, którego odległość Levensteina jest najmniejsza. Czy udało się poprawić ten zaszumiony tekst?  Wykorzystaj funkcję edit_distance z nltk.

In [68]:
# Dystans levensteina
from nltk.metrics.distance import edit_distance
from nltk import word_tokenize
from timeit import default_timer as timer  # funkcja potrzebna do mierzenia czasu

input_text = "dzięń dobry pańestwu, labolatolia pżeprowadzone bydą w sali omputerowej 1.6.10. pozdrawiem"

# zbiór słów, które są poprawnymi
with open('resources/slowa-min.txt', 'r', encoding='utf8') as f:
  valid_words = set([w.strip() for w in f.read().split("\n") if w.strip() != ''])

def get_closest_word(token, valid_words):
  return min(valid_words, key=lambda x: edit_distance(token, x)) if token not in valid_words else token

# tokenizuj
tokenized_input = word_tokenize(input_text)
# pobierz czas
start = timer()
# lista poprawionych tokenów
corrected = []
for token in tokenized_input:
  # jeśli token nie jest słowem lub jest już w słowniku — nie poprawiaj
  if not token.isalpha() or token in valid_words:
    corrected.append(token)
  else:
    # popraw
    corrected.append(get_closest_word(token, valid_words))
end = timer()

print(f"Tekst oryginalny: {input_text}")
print(f"Tekst poprawiony: {' '.join(corrected)}")
print(f"Czas poprawiania: {end - start}")

Tekst oryginalny: dzięń dobry pańestwu, labolatolia pżeprowadzone bydą w sali omputerowej 1.6.10. pozdrawiem
Tekst poprawiony: dzień dobry państwu , laboratoria przeprowadzone będą w sali komputerowej 1.6.10. pozdrawiam
Czas poprawiania: 1.5073075000000244


# Trenowanie własnych wektorów
Ponieważ trenowanie wektorów na dużym korpusie może być czasochłonne i wymaga dużego korpusu, aby wyłapać odpowiednie konteksty słów, nie wykonywaliśmy go na laboratoriach. Zainteresowanym tworzeniem własnych wektorów polecam artykuł: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/. Który opisuje jak wytrenować embeddingi z użyciem pythona i paczki "gensim".